In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import re
def clean_file(code):
  #no multiline comments
  clean_code=re.sub(r'/\*(.*?)\*/', '', code, flags=re.DOTALL)
  #no tabs
  clean_code=re.sub(r'\t',' ',clean_code)
  #no empty lines
  clean_code = re.sub(r'\n(\s*)\n', '\n', clean_code)
  clean_code= re.sub(r'^\n+','',clean_code)
  #lines of code
  matches=re.findall('\n',clean_code)
  return clean_code,len(matches)

In [3]:
file=open("/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-01/original/T1.java")
og,og_linesOfCode=clean_file(file.read())
print(og)
print(og_linesOfCode)

public class T1 {
 public static void main(String[] args) {
  System.out.println("Welcome to Java");
  System.out.println("Welcome to Java");
  System.out.println("Welcome to Java");
  System.out.println("Welcome to Java");
  System.out.println("Welcome to Java");
 }
}

9


In [4]:
file=open("/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-01/plagiarized/L1/03/Main.java")
plag,plag_linesOfCode=clean_file(file.read())
print(plag)
print(plag_linesOfCode)

public class Main {
    public static void main(String[] args) { 
            System.out.println("Welcome to Java"); 
            System.out.println("Welcome to Java"); 
            System.out.println("Welcome to Java"); 
            System.out.println("Welcome to Java"); 
            System.out.println("Welcome to Java");
    }
}

9


In [5]:
file=open("/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-01/non-plagiarized/03/Nomor1_2451041557A22145B3701B0184109CAB013.java",'r')
nonPlag,nonPlag_linesOfCode=clean_file(file.read())
print(nonPlag)
print(nonPlag_linesOfCode)

public class Nomor1_2451041557A22145B3701B0184109CAB013 {
    public static void main(String[] args) {
        for(int i=0;i<5;i++)
        {
            System .out.println("Welcome To Java");
        }
    }
}

8


In [6]:
import re

def count_loops(code):
    # Regular expression pattern to match loop structures
    loop_pattern = r'(?:for|while|do\s*while)'

    # Count the number of loop occurrences in the Java code
    loop_count = len(re.findall(loop_pattern, code))

    return loop_count

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def cosine_sim(code1,code2):
  data=[]
  data.append(re.sub(r'\n',' ',code1))
  data.append(re.sub(r'\n',' ',code2))
  vectorizer=TfidfVectorizer()
  X=vectorizer.fit_transform(data)
  tfidf_tokens=vectorizer.get_feature_names_out()
  df_tfidfvect = pd.DataFrame(data = X.toarray(),index = ['Doc1','Doc2'],columns = tfidf_tokens)
  return(cosine_similarity(X[0].toarray(),X[1].toarray()))

In [8]:
print(count_loops(og))
print(count_loops(plag))
print(count_loops(nonPlag))

0
0
1


In [9]:
print(cosine_sim(og,plag))
print(cosine_sim(og,nonPlag))

[[0.99084936]]
[[0.67120608]]


Importing files

In [48]:
import glob

files = glob.glob('/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/*', recursive = True)

for file in files:
  original=glob.glob(file+'/original/*', recursive = True)
  #non_plag_code stores required file location

  files1=glob.glob(file+'/*', recursive = True)
  for cases in files1:
    if cases.endswith('non-plagiarized'):
      non_plag=glob.glob(cases+'/*', recursive = True)
      for non_plag_folders in non_plag:
        non_plag_code=glob.glob(non_plag_folders+'/*',recursive=True)
        #non_plag_code stores required file location
        

    elif cases.endswith('plagiarized'):
      plagiarized=glob.glob(cases+'/*', recursive = True)
      for plagiarized_folders in plagiarized:
        plagiarized_code_parent_folder=glob.glob(plagiarized_folders+'/*', recursive = True)
        for plagiarized_code_folder in plagiarized_code_parent_folder:
          plagiarized_code=glob.glob(plagiarized_code_folder+'/*', recursive = True)
          #plagiarized_code stores required file location

In [ ]:

for file in files:
  plagiarized=glob.glob(file+'/plagiarized/*', recursive = True)
  for plagiarized_folders in plagiarized:
    plagiarized_code_parent_folder=glob.glob(plagiarized_folders+'/*', recursive = True)
    for plagiarized_code_folder in plagiarized_code_parent_folder:
      plagiarized_code=glob.glob(plagiarized_code_folder+'/*', recursive = True)
      print(plagiarized_code)
      #plagiarized_code stores required file location

In [60]:
import glob

files = glob.glob('/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/*', recursive = True)
c1=0
c2=0
c3=0
for file in files:

  original=glob.glob(file+'/original/*', recursive = True)
  #non_plag_code stores required file location
  c1+=1
  non_plag=glob.glob(file+'/non-plagiarized/*', recursive = True)
  for non_plag_folders in non_plag:
      non_plag_code=glob.glob(non_plag_folders+'/*',recursive=True)
      #non_plag_code stores required file location
      c2+=1
  plagiarized=glob.glob(file+'/plagiarized/*', recursive = True)
  for plagiarized_folders in plagiarized:
    plagiarized_code_parent_folder=glob.glob(plagiarized_folders+'/*', recursive = True)
    for plagiarized_code_folder in plagiarized_code_parent_folder:
      plagiarized_code=glob.glob(plagiarized_code_folder+'/*', recursive = True)
      c3+=1
      #plagiarized_code stores required file location


7 , 105 , 355


In [ ]:
def buildingDf(original_path,code_path,classification):
  


In [51]:
import pandas as pd

df=pd.DataFrame(columns=['Diff in LOC','Diff in No. of Loops','Cosine_similarity','Class'])

In [52]:
df

,Diff in LOC,Diff in No. of Loops,Cosine_similarity,Class
